In [1]:
import os
import pandas as pd
from dotenv import load_dotenv, find_dotenv; _ = load_dotenv(find_dotenv())

In [2]:
topic_type = 'foods'
topic_file = f'{topic_type}.txt'

In [3]:
df = pd.read_csv(f'data/{topic_type}/data.csv',index_col=0, dtype=str)
df['embedding_2'] = [eval(e) for e in df['embedding_2']]
df['embedding_3_small'] = [eval(e) for e in df['embedding_3_small']]
df['embedding_3_large'] = [eval(e) for e in df['embedding_3_large']]
print(len(df))
df.head(1)

450


,topic,lang,format,length,text,embedding_2,embedding_3_small,embedding_3_large
0,じゃがいも,en,prose,long_sum,Potatoes are a vegetable native to South Ameri...,"[0.017380426068302725, -0.018543428196079303, ...","[-0.006245624336255771, -0.04292074686214286, ...","[0.012517579007594287, -0.04335306299595459, -..."


In [4]:
import umap
import plotly.express as px

params = {
    'n_neighbors': 400,
    'min_dist': 0.2,
    'metric': 'cosine',
    # 'metric': 'euclidean',
}

In [5]:
embeddings_2 = df['embedding_2'].tolist()
embeddings_3_small = df['embedding_3_small'].tolist()
embeddings_3_large = df['embedding_3_large'].tolist()

In [6]:
umap_model = umap.UMAP(n_components=2, **params)
embeddings_2_2d = umap_model.fit_transform(embeddings_2)
embeddings_3s_2d = umap_model.fit_transform(embeddings_3_small)
embeddings_3l_2d = umap_model.fit_transform(embeddings_3_large)

In [7]:
type(embeddings_2_2d)

numpy.ndarray

In [8]:
for i,id in enumerate(df.index):
    df.loc[id,'x_2'] = embeddings_2_2d[i][0]
    df.loc[id,'y_2'] = embeddings_2_2d[i][1]
    df.loc[id,'x_3s'] = embeddings_3s_2d[i][0]
    df.loc[id,'y_3s'] = embeddings_3s_2d[i][1]
    df.loc[id,'x_3l'] = embeddings_3l_2d[i][0]
    df.loc[id,'y_3l'] = embeddings_3l_2d[i][1]
df.to_csv('all_data.csv')

In [9]:
os.makedirs(f"output/{topic_type}", exist_ok=True)
def save(embedding_model, color):
    x = ''
    y = ''
    if embedding_model == '2':
        x = 'x_2'
        y = 'y_2'
    elif embedding_model == '3s':
        x = 'x_3s'
        y = 'y_3s'
    elif embedding_model == '3l':
        x = 'x_3l'
        y = 'y_3l'
    else:
        print('Error')
    fig = px.scatter(
        df, x=x, y=y,
        color=color,
        hover_data=['topic','lang','format','length'],
        opacity=0.5
    )
    fig.write_html(f'output/{topic_type}/{embedding_model}_{color}_scatter.html')
    fig.write_image(f'output/{topic_type}/{embedding_model}_{color}_scatter.png')


### ada-002

#### 食材

In [10]:
save('2','topic')

#### 形式

In [11]:
save('2','format')

### 言語

In [12]:
save('2','lang')

### 3-small

In [13]:
save('3s','topic')

In [14]:
save('3s','format')

In [15]:
save('3s','lang')

### 3-large

In [16]:
save('3l','topic')

In [17]:
save('3l','format')

In [18]:
save('3l','lang')